# Amazon Bedrock Application Inference Profile

이 노트북은 조직이 Amazon Bedrock 애플리케이션 추론 프로파일을 구현, 테스트, 검증 및 운영하는 방법을 보여줍니다. 애플리케이션 추론 프로파일을 효과적으로 관리하고 활용하는 방법에 대한 포괄적인 이해를 제공하는 것이 목표입니다.

# Overview

아마존 베드락 애플리케이션 추론 프로파일을 통해 조직은 모든 베드락 기반 파운데이션 모델에 비용 할당 태그를 지정하여 비용 센터, 사업 단위, 팀, 애플리케이션과 같은 조직 분류 체계별로 사용량을 분류할 수 있습니다. 여러 워크로드에서 AI 지출을 관리하는 이 확장 가능한 프로그래밍 방식은 수동 프로세스에 대한 의존도를 낮추고 비용 초과 위험을 줄이며 중요한 애플리케이션이 우선순위를 갖도록 보장합니다. AI 관련 비용에 대한 향상된 가시성과 제어 기능을 통해 조직은 GenAI 투자를 최적화하고 보다 효율적으로 혁신을 추진할 수 있습니다. 이 노트북은 애플리케이션 추론 프로필을 생성하고 모델을 호출하는 데 사용하는 방법을 보여줍니다.

# Architecture
![Architecture](./images/architecture.png)

Amazon Bedrock의 추론 프로파일은 두 가지 유형이 있습니다:
1. 크로스 리전 추론 프로파일
- Bedrock 서비스가 사전 정의
- 여러 리전으로 모델 요청을 라우팅
- 복원력 향상과 처리량 개선
- 트래픽 급증 관리에 효과적

2. 애플리케이션 추론 프로파일
- 사용자가 직접 생성
- 단일 리전 또는 여러 리전으로 라우팅 가능
- 비용과 모델 사용량 추적

주요 이점:
- CloudWatch 로그를 통한 사용량 지표 추적
- 태그 기반 비용 모니터링
- 크로스 리전 추론으로 처리량 향상

이 노트북은 애플리케이션 추론 프로파일을 생성하고 이를 모델 호출에 활용하는 방법을 보여줍니다.

# Use case

이 노트북에는 다음과 같은 다양한 Amazon Bedrock 애플리케이션 추론 프로파일 API 기능에 대한 구현, 테스트 및 유효성 검사 단계가 포함되어 있습니다:
* Create Inference Profile
* Get Inference Profile
* List Inference Profiles
* Invoke model with Inference Profile using Converse API
* Invoke model with Inference Profile using ConverseStream API
* Invoke model with Inference Profile using InvokeModel API
* Invoke model with Inference Profile using InvokeModelWithResponseStream API
* Tag Resource
* List Tags For Resource
* Untag Resource
* Cleanup


## Prerequisites

1. AWS 계정이 생성되었는지 확인합니다.: [Link](https://docs.aws.amazon.com/accounts/latest/reference/manage-acct-creating.html)
2. 사용자에게 Amazon Bedrock에서 올바른 모델에 액세스할 수 있는 권한이 있는지 확인합니다.: [Link](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
3. 이 노트북은 AWS 자격 증명을 설정하여 진행했습니다.

## Notebook Code with Comments

### Install dependecies

In [ ]:
!pip install --upgrade --force-reinstall boto3 botocore awscli

In [1]:
import boto3
print(boto3.__version__)

1.35.91


In [2]:
import boto3
import sagemaker
import re
import time
import json

session = boto3.Session()
sagemaker_session = sagemaker.Session()
#role = sagemaker.get_execution_role()
current_region = sagemaker_session.boto_region_name 

bedrock = session.client("bedrock", region_name=current_region)
br = session.client("bedrock-runtime", region_name=current_region)

/Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/06/25 20:39:31] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=685847;file:///Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=714194;file:///Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/younghwa/Library/Application Support/sagemaker/config.yaml


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=825676;file:///Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=488867;file:///Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=368407;file:///Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=977657;file:///Users/younghwa/Desktop/chloe/LAB/Bedrock-Features/application_profile/infp/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

In [3]:
current_region

'us-east-1'

### List application inference profiles

In [4]:
bedrock.list_inference_profiles(typeEquals='APPLICATION')

{'ResponseMetadata': {'RequestId': '0db2462b-2142-4d2a-8301-ad05d6cda643',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Jan 2025 11:39:47 GMT',
   'content-type': 'application/json',
   'content-length': '31416',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0db2462b-2142-4d2a-8301-ad05d6cda643'},
  'RetryAttempts': 0},
 'inferenceProfileSummaries': [{'inferenceProfileName': 'dzd_60k0gby0fv0wh3',
   'description': 'Created by Amazon SageMaker Unified Studio for domain dzd_60k0gby0fv0wh3 to provide access to Amazon Bedrock model in domain playgrounds',
   'createdAt': datetime.datetime(2024, 12, 16, 14, 36, 22, 326831, tzinfo=tzutc()),
   'updatedAt': datetime.datetime(2024, 12, 16, 14, 36, 22, 326831, tzinfo=tzutc()),
   'inferenceProfileArn': 'arn:aws:bedrock:us-east-1:603420654815:application-inference-profile/a5o6k9xmzuoy',
   'models': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-mid-v1'}],
   'inferenceProfileId': 'a5o6k9xmzuoy',
   's

### List cross region inference profiles

In [5]:
bedrock.list_inference_profiles(typeEquals='SYSTEM_DEFINED')

{'ResponseMetadata': {'RequestId': 'ed755479-9e3e-4df2-8d5f-16b5d6bddcc5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Jan 2025 11:40:50 GMT',
   'content-type': 'application/json',
   'content-length': '10883',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ed755479-9e3e-4df2-8d5f-16b5d6bddcc5'},
  'RetryAttempts': 0},
 'inferenceProfileSummaries': [{'inferenceProfileName': 'US Anthropic Claude 3 Sonnet',
   'description': 'Routes requests to Anthropic Claude 3 Sonnet in us-east-1 and us-west-2.',
   'createdAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzutc()),
   'updatedAt': datetime.datetime(2024, 8, 26, 0, 0, tzinfo=tzutc()),
   'inferenceProfileArn': 'arn:aws:bedrock:us-east-1:603420654815:inference-profile/us.anthropic.claude-3-sonnet-20240229-v1:0',
   'models': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'},
    {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-

In [6]:
models = [{'modelArn': fm['modelArn'], 'modelName':fm['modelName'], 'modelId': fm['modelId']} for fm in bedrock.list_foundation_models()['modelSummaries'] if fm['modelName'].startswith('Nova') and fm['modelLifecycle']['status']== 'ACTIVE']
models

[{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0:300k',
  'modelName': 'Nova Pro',
  'modelId': 'amazon.nova-pro-v1:0:300k'},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0',
  'modelName': 'Nova Pro',
  'modelId': 'amazon.nova-pro-v1:0'},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0:300k',
  'modelName': 'Nova Lite',
  'modelId': 'amazon.nova-lite-v1:0:300k'},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-lite-v1:0',
  'modelName': 'Nova Lite',
  'modelId': 'amazon.nova-lite-v1:0'},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-canvas-v1:0',
  'modelName': 'Nova Canvas',
  'modelId': 'amazon.nova-canvas-v1:0'},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-reel-v1:0',
  'modelName': 'Nova Reel',
  'modelId': 'amazon.nova-reel-v1:0'},
 {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-micro-v1:

## Setup single Region application inference profile

단일 리전에 대한 애플리케이션 추론 프로필을 생성하려면 기반 모델의 ARN을 지정해야 합니다. 해당 리전의 모델에 대한 요청의 사용량과 비용이 추적됩니다. 요청 생성 시 다음 매개변수를 제공해야 합니다:

* inferenceProfileName - 추론 프로필의 이름
* modelSource - 단일 리전의 경우 copyFrom 속성에 기반 모델의 ARN을 지정
* description - 추론 프로필에 대한 설명 (선택사항)
* tags - 추론 프로필에 태그 첨부. AWS 비용 할당 태그를 사용하여 비용 추적 가능. 프로젝트 ID, 부서 ID 또는 원하는 방식으로 비용 추적 가능

In [7]:
#Amazon Nova Pro
#Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0
#Model ID: amazon.nova-pro-v1:0

modelId = 'amazon.nova-pro-v1:0'
modelARN = [m['modelArn'] for m in models if m['modelId'] == modelId][0]
modelId, modelARN 

('amazon.nova-pro-v1:0',
 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0')

In [8]:
inf_profile_response = bedrock.create_inference_profile(
    inferenceProfileName='NovaProPart',
    description='Application profile for Nova Pro',
    modelSource={
        'copyFrom': modelARN
    },
    tags=[
        {
            'key': 'projectId',
            'value': 'parts2025'
        },
    ]
)

In [9]:
inf_profile_arn = inf_profile_response['inferenceProfileArn']
inf_profile_arn

'arn:aws:bedrock:us-east-1:603420654815:application-inference-profile/w19uuvvsn8b2'

In [10]:
bedrock.get_inference_profile(inferenceProfileIdentifier=inf_profile_arn)

{'ResponseMetadata': {'RequestId': '32033ee1-aa21-4c18-b3bf-d578a0030699',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Jan 2025 11:41:32 GMT',
   'content-type': 'application/json',
   'content-length': '449',
   'connection': 'keep-alive',
   'x-amzn-requestid': '32033ee1-aa21-4c18-b3bf-d578a0030699'},
  'RetryAttempts': 0},
 'inferenceProfileName': 'NovaProPart',
 'description': 'Application profile for Nova Pro',
 'createdAt': datetime.datetime(2025, 1, 6, 11, 41, 20, 242364, tzinfo=tzutc()),
 'updatedAt': datetime.datetime(2025, 1, 6, 11, 41, 20, 242364, tzinfo=tzutc()),
 'inferenceProfileArn': 'arn:aws:bedrock:us-east-1:603420654815:application-inference-profile/w19uuvvsn8b2',
 'models': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0'}],
 'inferenceProfileId': 'w19uuvvsn8b2',
 'status': 'ACTIVE',
 'type': 'APPLICATION'}

In [11]:
inf_profile_id = bedrock.get_inference_profile(inferenceProfileIdentifier=inf_profile_arn)['inferenceProfileId']
inf_profile_id

'w19uuvvsn8b2'

### Example usage with Converse API

추론 프로파일을 사용하려면 modelId 필드에 추론 프로파일의 ARN을 지정합니다. 

In [36]:
from time import time
from pprint import pprint
system_prompt = "You are an expert on AWS services and always provide correct and concise answers."
input_message = "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요??"
start = time()
response = br.converse(
    modelId=inf_profile_arn,
    system=[{"text": system_prompt}],
    messages=[{
        "role": "user",
        "content": [{"text": input_message}]
    }]
)
end = time()

print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response['output']['message']['content']}")


Response time: 12 second(s)
Using Application Inf Profile::Response output: [{'text': '비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS 중 어떤 것을 사용할지는 주로 애플리케이션의 요구 사항에 따라 결정됩니다. 다음은 각각의 서비스에 대한 주요 특징과 비용 고려 사항입니다.\n\n### Amazon S3\n- **용도**: 객체 저장소로, 대용량 데이터의 저장 및 검색에 적합합니다.\n- **특징**:\n  - 높은 내구성과 가용성\n  - 다양한 스토리지 클래스 지원 (Standard, Infrequent Access, Glacier 등)\n  - 간단한 데이터 관리 및 액세스 제어\n- **비용**:\n  - 저장 용량, 데이터 전송, 요청 수 등에 따라 비용 발생\n  - Infrequent Access 및 Glacier와 같은 저렴한 스토리지 클래스를 사용하면 비용 절감 가능\n\n### Amazon EFS\n- **용도**: 분산 파일 시스템으로, 여러 Amazon EC2 인스턴스에서 공유 파일 시스템을 필요로 할 때 사용합니다.\n- **특징**:\n  - 네트워크 파일 시스템 (NFS) 프로토콜 지원\n  - 동적으로 확장 가능\n  - 실시간 공유 및 동기화\n- **비용**:\n  - 저장 용량, 스루풋, 데이터 전송 등에 따라 비용 발생\n  - 주로 빈번한 액세스가 필요한 경우에 비용이 더 높을 수 있음\n\n### 결론\n- **Amazon S3**: 주로 데이터가 빈번하게 액세스되지 않거나, 대용량 저장이 필요한 경우에 비용 효율적입니다. 특히, 데이터의 액세스 패턴에 따라 적절한 스토리지 클래스를 선택하면 더 큰 비용 절감 효과를 볼 수 있습니다.\n- **Amazon EFS**: 여러 EC2 인스턴스에서 공유되는 파일 시스템이 필요하고, 빈번한 액세스가 필요한 경우에 적합합니다. 하지만, 비용 측면에서는 S3에 비해 더 높을 수 있습

In [17]:
import re

text = response['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS 중 하나를 선택할 때는 용도와 요구 사항에 따라 다릅니다. 각각의 특성을 고려하여 결정해야 합니다.

### Amazon S3
- 용도: 객체 저장 (파일, 이미지, 비디오 등)
- 비용 효율성: 
  - 다양한 스토리지 클래스 (S3 Standard, S3 Intelligent-Tiering, S3 Standard-Infrequent Access, S3 One Zone-Infrequent Access, S3 Glacier, S3 Glacier Deep Archive)를 통해 비용 최적화 가능.
  - 데이터 액세스 빈도에 따라 적절한 스토리지 클래스 선택 가능.
- 장점:
  - 높은 내구성과 가용성.
  - 스케일아웃이 쉽고, 관리 부담이 적음.
  - 버킷 정책과 라이프사이클 관리를 통한 자동화 가능.

### Amazon EFS
- 용도: 분산 파일 시스템 (애플리케이션의 공유 파일 시스템)
- 비용 효율성:
  - 실시간 액세스가 필요한 경우에 적합하지만, 비용은 저장된 데이터 양과 처리량에 따라 달라짐.
  - 장기 보관에 비해 운영 중인 파일 시스템에 더 적합.
- 장점:
  - 여러 EC2 인스턴스에서 공유 파일 시스템을 사용할 수 있음.
  - 실시간 액세스와 낮은 레이턴시 제공.
  - 관리가 간편함.

### 결론
- 문서 저장이 주 목적이고, 장기 보관이나 빈번한 액세스가 필요하지 않다면 Amazon S3가 더 비용 효율적입니다.
- 실시간 액세스가 필요하고, 여러 인스턴스에서 공유되는 파일 시스템이 필요하다면 Amazon EFS를 선택하는 것이 좋습니다.

각각의 요구 사항에 맞게 서비스를 선택하는 것이 중요합니다.


### Example usage with invoke model

In [23]:
import json
system_list = [
            {
                "text": "You are an expert on AWS services and always provide correct and concise answers"
            }
]
message_list = [{"role": "user", "content": [{"text": "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요??"}]}]
inf_params = {"max_new_tokens": 500, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_list,
    "messages":message_list,
    "inferenceConfig" : inf_params,
}
)

accept = 'application/json'
contentType = 'application/json'
start = time()
response = br.invoke_model(body=body, modelId=inf_profile_arn, accept=accept, contentType=contentType)
end = time()
response_body = json.loads(response.get('body').read())

print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response_body['output']['message']['content']}")

Response time: 6 second(s)
Using Application Inf Profile::Response output: [{'text': '비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS 중 어떤 것을 사용할지 결정할 때는 다음과 같은 요소들을 고려해야 합니다:\n\n### Amazon S3\n- **용도**: 객체 저장소로, 대용량의 비정형 데이터(문서, 미디어 파일 등) 저장에 적합.\n- **비용**:\n  - 저장 비용이 상대적으로 저렴함.\n  - 데이터 전송 비용이 발생할 수 있음.\n- **스케일러빌리티**: 무제한으로 확장 가능.\n- **접근 속도**: 높은 대기 시간을 가질 수 있음 (몇 백 밀리초).\n- **데이터 일관성**: 최종 일관성 모델을 따름.\n\n### Amazon EFS\n- **용도**: 분산 파일 시스템으로, 여러 인스턴스에서 공유 파일 시스템을 필요로 할 때 사용.\n- **비용**:\n  - 저장 비용이 S3보다 비쌈.\n  - 네트워크 처리 비용 및 처리량 비용 추가 발생.\n- **스케일러빌리티**: 확장 가능하지만 S3만큼 무제한은 아님.\n- **접근 속도**: 낮은 대기 시'}]


In [25]:
import re

text = response_body['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS 중 어떤 것을 사용할지 결정할 때는 다음과 같은 요소들을 고려해야 합니다:

### Amazon S3
- 용도: 객체 저장소로, 대용량의 비정형 데이터(문서, 미디어 파일 등) 저장에 적합.
- 비용:
  - 저장 비용이 상대적으로 저렴함.
  - 데이터 전송 비용이 발생할 수 있음.
- 스케일러빌리티: 무제한으로 확장 가능.
- 접근 속도: 높은 대기 시간을 가질 수 있음 (몇 백 밀리초).
- 데이터 일관성: 최종 일관성 모델을 따름.

### Amazon EFS
- 용도: 분산 파일 시스템으로, 여러 인스턴스에서 공유 파일 시스템을 필요로 할 때 사용.
- 비용:
  - 저장 비용이 S3보다 비쌈.
  - 네트워크 처리 비용 및 처리량 비용 추가 발생.
- 스케일러빌리티: 확장 가능하지만 S3만큼 무제한은 아님.
- 접근 속도: 낮은 대기 시


## 다중 리전 애플리케이션 추론 프로파일
리전 간 애플리케이션 추론 프로파일을 생성하려면 크로스 리전 추론 프로파일의 ARN을 지정하고, 나머지 매개변수는 단일 리전 애플리케이션 추론 프로파일과 동일합니다.

* inferenceProfileName - 추론 프로파일의 이름
* modelSource - 다중 리전 애플리케이션 프로파일의 경우 copyFrom 속성에 크로스 리전(시스템 정의) 추론 프로파일의 ARN을 지정
* description - 추론 프로파일에 대한 설명 (선택사항)
* tags - 추론 프로파일에 태그 첨부. AWS 비용 할당 태그를 사용하여 비용 추적 가능. 프로젝트 ID, 부서 ID 또는 원하는 방식으로 비용 추적 가능

In [26]:
cr_inf_profile = [ip for ip in bedrock.list_inference_profiles(typeEquals='SYSTEM_DEFINED')['inferenceProfileSummaries'] if ip['inferenceProfileName'] == 'US Nova Pro'][0]
cr_inf_profile

{'inferenceProfileName': 'US Nova Pro',
 'description': 'Routes requests to Nova Pro in us-east-1, us-west-2 and us-east-2.',
 'createdAt': datetime.datetime(2024, 11, 29, 13, 23, tzinfo=tzutc()),
 'updatedAt': datetime.datetime(2024, 11, 29, 13, 23, tzinfo=tzutc()),
 'inferenceProfileArn': 'arn:aws:bedrock:us-east-1:603420654815:inference-profile/us.amazon.nova-pro-v1:0',
 'models': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-pro-v1:0'},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-pro-v1:0'},
  {'modelArn': 'arn:aws:bedrock:us-east-2::foundation-model/amazon.nova-pro-v1:0'}],
 'inferenceProfileId': 'us.amazon.nova-pro-v1:0',
 'status': 'ACTIVE',
 'type': 'SYSTEM_DEFINED'}

In [27]:
cr_inf_profile_arn = 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-pro-v1:0'
cr_app_inf_profile_response = bedrock.create_inference_profile(
    inferenceProfileName='NovaProSupplyCrossRegion',
    description='Application profile for Nova Pro with cross regional routing',
    modelSource={
        'copyFrom': cr_inf_profile_arn
    },
    tags=[
        {
            'key': 'projectId',
            'value': 'supply2026'
        },
    ]
)

In [28]:
cr_app_inf_profile_arn = cr_app_inf_profile_response['inferenceProfileArn']
cr_app_inf_profile_arn

'arn:aws:bedrock:us-east-1:603420654815:application-inference-profile/5u28hty6yb03'

In [29]:
bedrock.get_inference_profile(inferenceProfileIdentifier=cr_app_inf_profile_arn)

{'ResponseMetadata': {'RequestId': 'a6c92713-01de-437a-a00b-53e5f8fd41fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Jan 2025 11:46:38 GMT',
   'content-type': 'application/json',
   'content-length': '490',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a6c92713-01de-437a-a00b-53e5f8fd41fb'},
  'RetryAttempts': 0},
 'inferenceProfileName': 'NovaProSupplyCrossRegion',
 'description': 'Application profile for Nova Pro with cross regional routing',
 'createdAt': datetime.datetime(2025, 1, 6, 11, 46, 30, 716133, tzinfo=tzutc()),
 'updatedAt': datetime.datetime(2025, 1, 6, 11, 46, 30, 716133, tzinfo=tzutc()),
 'inferenceProfileArn': 'arn:aws:bedrock:us-east-1:603420654815:application-inference-profile/5u28hty6yb03',
 'models': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-pro-v1:0'}],
 'inferenceProfileId': '5u28hty6yb03',
 'status': 'ACTIVE',
 'type': 'APPLICATION'}

### Example usage Converse API

In [30]:
from time import time
system_prompt = "You are an expert on AWS services and always provide correct and concise answers."
input_message = "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요?"

start = time()
response = br.converse(
    modelId=cr_app_inf_profile_arn,
    system=[{"text": system_prompt}],
    messages=[{
        "role": "user",
        "content": [{"text": input_message}]
    }]
)
end = time()
print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response['output']['message']['content']}")

Response time: 9 second(s)
Using Application Inf Profile::Response output: [{'text': '비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS를 비교할 때 다음과 같은 요소들을 고려해야 합니다:\n\n### Amazon S3\n- **용도**: 객체 저장소로, 대용량의 비정형 데이터 (문서, 미디어 파일 등)를 저장하는 데 적합합니다.\n- **비용**: 저장 비용이 상대적으로 저렴합니다. 자주 접근하지 않는 데이터의 경우 S3 Standard-IA나 S3 Glacier와 같은 저비용 스토리지 클래스를 사용할 수 있습니다.\n- **스케일**: 거의 무제한으로 확장 가능합니다.\n- **접근 속도**: 네트워크 지연 시간이 다소 길 수 있습니다.\n\n### Amazon EFS\n- **용도**: 분산 파일 시스템으로, 여러 EC2 인스턴스에서 공유 파일 시스템을 필요로 할 때 적합합니다.\n- **비용**: 저장 비용이 S3보다 높을 수 있습니다. 또한, 처음 1개월은 무료이지만 이후에는 스루풋과 관련된 비용이 추가됩니다.\n- **스케일**: 수백 GB에서 수 페타바이트까지 확장 가능합니다.\n- **접근 속도**: 네트워크 파일 시스템으로 접근 속도가 빠릅니다.\n\n### 결론\n- **주로 읽기 중심의 대용량 데이터 저장**: Amazon S3가 더 비용 효율적입니다.\n- **여러 인스턴스에서 공유되는 파일 시스템 필요**: Amazon EFS가 더 적합할 수 있지만 비용은 높을 수 있습니다.\n\n따라서, 문서를 단순히 저장하고 관리하는 경우에는 Amazon S3를, 여러 인스턴스에서 공유하고 빠른 접근이 필요한 경우에는 Amazon EFS를 고려하는 것이 좋습니다.'}]


In [31]:
import re

text = response['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS를 비교할 때 다음과 같은 요소들을 고려해야 합니다:

### Amazon S3
- 용도: 객체 저장소로, 대용량의 비정형 데이터 (문서, 미디어 파일 등)를 저장하는 데 적합합니다.
- 비용: 저장 비용이 상대적으로 저렴합니다. 자주 접근하지 않는 데이터의 경우 S3 Standard-IA나 S3 Glacier와 같은 저비용 스토리지 클래스를 사용할 수 있습니다.
- 스케일: 거의 무제한으로 확장 가능합니다.
- 접근 속도: 네트워크 지연 시간이 다소 길 수 있습니다.

### Amazon EFS
- 용도: 분산 파일 시스템으로, 여러 EC2 인스턴스에서 공유 파일 시스템을 필요로 할 때 적합합니다.
- 비용: 저장 비용이 S3보다 높을 수 있습니다. 또한, 처음 1개월은 무료이지만 이후에는 스루풋과 관련된 비용이 추가됩니다.
- 스케일: 수백 GB에서 수 페타바이트까지 확장 가능합니다.
- 접근 속도: 네트워크 파일 시스템으로 접근 속도가 빠릅니다.

### 결론
- 주로 읽기 중심의 대용량 데이터 저장: Amazon S3가 더 비용 효율적입니다.
- 여러 인스턴스에서 공유되는 파일 시스템 필요: Amazon EFS가 더 적합할 수 있지만 비용은 높을 수 있습니다.

따라서, 문서를 단순히 저장하고 관리하는 경우에는 Amazon S3를, 여러 인스턴스에서 공유하고 빠른 접근이 필요한 경우에는 Amazon EFS를 고려하는 것이 좋습니다.


### Example usage Invoke Model

In [34]:
import json
system_list = [
            {
                "text": "You are an expert on AWS services and always provide correct and concise answers"
            }
]
message_list = [{"role": "user", "content": [{"text": "비용 효율적인 애플리케이션을 구성하기 위해서는 Amazon S3 또는 EFS에 문서를 저장하는게 좋을까요??"}]}]
inf_params = {"max_new_tokens": 500, "top_p": 0.9, "top_k": 20, "temperature": 0.7}

body = json.dumps({
    "schemaVersion": "messages-v1",
    "system": system_list,
    "messages":message_list,
    "inferenceConfig" : inf_params,
}
)
accept = 'application/json'
contentType = 'application/json'
start = time()
response = br.invoke_model(body=body, modelId=cr_app_inf_profile_arn, accept=accept, contentType=contentType)
end = time()
response = json.loads(response.get('body').read())

print(f"Response time: {int(end-start)} second(s)")
print(f"Using Application Inf Profile::Response output: {response['output']['message']['content']}")



Response time: 6 second(s)
Using Application Inf Profile::Response output: [{'text': '비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS 중 어떤 것을 사용할지는 여러 요인에 따라 달라집니다. 각각의 특성을 이해하고 요구사항에 맞게 선택하는 것이 중요합니다.\n\n### Amazon S3\n- **용도**: 객체 저장소로, 대규모의 비정형 데이터 (문서, 비디오, 이미지 등)를 저장하는 데 적합합니다.\n- **비용**: 저장 비용이 상대적으로 저렴합니다. 또한, 데이터의 액세스 패턴에 따라 다른 스토리지 클래스 (Standard, Infrequent Access, Glacier 등)를 선택할 수 있습니다.\n- **스케일**: 거의 무한한 스케일 아웃이 가능합니다.\n- **접근성**: 객체는 HTTP(S)를 통해 전 세계에서 접근할 수 있습니다.\n- **관리**: 관리 오버헤드가 낮습니다. AWS가 백엔드를 관리합니다.\n\n### Amazon EFS\n- **용도**: 탄력적 파일 시스템으로, 여러 Amazon EC2 인스턴스에서 공유 파일 시스템을 필요로 할 때 사용합니다.\n- **비용**: 스토리지 비용이 S3보다 비쌀 수 있습니다. 또한,'}]


In [35]:
import re

text = response['output']['message']['content'][0]['text']
clean_text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
formatted_text = clean_text.strip()
print(formatted_text)

비용 효율적인 애플리케이션을 구성하기 위해 Amazon S3와 Amazon EFS 중 어떤 것을 사용할지는 여러 요인에 따라 달라집니다. 각각의 특성을 이해하고 요구사항에 맞게 선택하는 것이 중요합니다.

### Amazon S3
- 용도: 객체 저장소로, 대규모의 비정형 데이터 (문서, 비디오, 이미지 등)를 저장하는 데 적합합니다.
- 비용: 저장 비용이 상대적으로 저렴합니다. 또한, 데이터의 액세스 패턴에 따라 다른 스토리지 클래스 (Standard, Infrequent Access, Glacier 등)를 선택할 수 있습니다.
- 스케일: 거의 무한한 스케일 아웃이 가능합니다.
- 접근성: 객체는 HTTP(S)를 통해 전 세계에서 접근할 수 있습니다.
- 관리: 관리 오버헤드가 낮습니다. AWS가 백엔드를 관리합니다.

### Amazon EFS
- 용도: 탄력적 파일 시스템으로, 여러 Amazon EC2 인스턴스에서 공유 파일 시스템을 필요로 할 때 사용합니다.
- 비용: 스토리지 비용이 S3보다 비쌀 수 있습니다. 또한,


# END OF NOTEBOOK 